In [ ]:
import pandas as pd

df = pd.read_excel('~/datasets/nlp/ori.xlsx')
df.fillna('NULL', inplace=True)
df.columns = ['id', 'content', 'subject', 'object', 'issue', 'platform', 'amount']
df['extract'] = df.apply(lambda x: f"抽取主体={x['subject']}, 抽取客体={x['object']}, 抽取问题描述={x['issue']}, 抽取平台={x['platform']}, 抽取金额={x['amount']}", axis=1)


lengths = df['extract'].str.len().tolist()
sorted(lengths, reverse=True)


In [12]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
from deep_translator import MyMemoryTranslator
ta = MyMemoryTranslator(source='zh-CN', target='en-US')
tb = MyMemoryTranslator(source='en-US', target='zh-CN')
import json

df = pd.read_excel('~/datasets/nlp/rev.xlsx')
df.fillna('NULL', inplace=True)
df.columns = ['id', 'content', 'subject', 'object', 'issue', 'platform', 'amount']
df['extract'] = df.apply(lambda x: f"抽取主体={x['subject']}, 抽取客体={x['object']}, 抽取问题描述={x['issue']}, 抽取平台={x['platform']}, 抽取金额={x['amount']}", axis=1)
prompt='你的任务是为接下来这段内容抽取[主体、客体、问题描述、平台、金额]，以#抽取主体=..., 抽取客体=..., 抽取问题描述=..., 抽取平台=..., 抽取金额=...#的格式给出，没有找到的内容请填上null'
case='例子：输入#刘爽投诉：2022年12月11日在淘宝平台仙桃市龙华山菜帮村湖北法保瑞防护科技有限公司n95口罩共计57.18元，商家承诺是48小时内发货，现商家迟迟不发货，请市监局调处！#，回答#抽取主体=湖北法保瑞防护科技有限公司, 抽取客体=口罩, 抽取问题描述=商家迟迟不发货, 抽取平台=淘宝, 抽取金额=57.18元#'

def random_insertion_deletion_replacement(text, n=5):
    words = text.split()
    for _ in range(n):
        if words:
            new_word = random.choice(words)
            index = random.randint(0, len(words))
            words.insert(index, new_word)
        if len(words) > 1:
            del words[random.randint(0, len(words) - 1)]
        if len(words) > 1:
            index1, index2 = random.randint(0, len(words) - 1), random.randint(0, len(words) - 1)
            words[index1], words[index2] = words[index2], words[index1]
    return ' '.join(words)

augmented_data = []
for i, row in df.iloc[500:1500].iterrows():
    if i%100==0:
        print(i)
    new_content = random_insertion_deletion_replacement(row['content'])
    augmented_data.append({'content': new_content, 'extract': row['extract']})

augmented_df = pd.DataFrame(augmented_data)
df = pd.concat([df, augmented_df], ignore_index=True)

print(df.shape)

df['question'] = prompt+'\n'+case+'\n输入内容如下#\n'+ df['content']

df = df[['question', 'extract']]  # 保留两列

def save_json_lines(df, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for record in df.to_dict(orient='records'):
            json_str = json.dumps(record, ensure_ascii=False)
            file.write(json_str + '\n')

save_json_lines(df, 'all.json')
a=df.iloc[:1500]
b = df.iloc[1500:5000]
c = df.iloc[5000:7000]
train_df, test_df = train_test_split(b, test_size=0.2, random_state=42)
train_df = pd.concat([a, train_df, c])

save_json_lines(train_df, 'train.json')
save_json_lines(test_df, 'test.json')

small_df = test_df.head(int(48))
save_json_lines(small_df, 'small.json')


500
600
700
800
900
1000
1100
1200
1300
1400
(6500, 8)


In [11]:
import os

def delete_json_files(directory):
    # 遍历指定目录
    for filename in os.listdir(directory):
        # 检查文件扩展名是否为.json
        if filename.endswith('.json'):
            # 构建完整的文件路径
            file_path = os.path.join(directory, filename)
            # 删除文件
            os.remove(file_path)
            print(f"Deleted: {file_path}")

# 指定你的目录路径
path_to_directory = '/data/yanghq/nlp_ptuning/pre/'

# 调用函数
delete_json_files(path_to_directory)

Deleted: /data/yanghq/nlp_ptuning/pre/all.json
Deleted: /data/yanghq/nlp_ptuning/pre/small.json
Deleted: /data/yanghq/nlp_ptuning/pre/train.json
Deleted: /data/yanghq/nlp_ptuning/pre/test.json
